In [ ]:
code source : github
présentation : architecture, modélisation, avantage et inconvenients;

modelisation de votre donnee; compromis, volumetrie en tant que stockage 


volumetrie 1 mois de données et répondre au 4 questions

### Liens importants

Pour tester les commandes sur google query en limitant le nombre de requête 
https://blog.gdeltproject.org/announcing-partitioned-gdelt-bigquery-tables/

des exemples de requêtes avec GKG
https://blog.gdeltproject.org/google-bigquery-gkg-2-0-sample-queries/

pour créer des timelines
https://blog.gdeltproject.org/getting-started-with-gdelt-google-cloud-datalab-simple-timelines/

des requêtes plus importantes

https://blog.gdeltproject.org/a-compilation-of-gdelt-bigquery-demos/




http://data.gdeltproject.org/documentation/ISA.2013.GDELT.pdf


## Objectif
une courte présentation de votre architecture, modélisation, les avantages et inconvénients, des choix de modélisation et d’architecture, volumétrie, limites et contraintes (max 10 slides de présentation)



## modélisation - question

du 1er juin 2016 au 10 juin 2016

### 1) afficher le nombre d’articles/évènements qu’il y a eu pour chaque triplet (jour, pays de l’évènement, langue de l’article).


GOOGLE QUERY 
-----------------
https://bigquery.cloud.google.com/table/gdelt-bq:gdeltv2.gkg_partitioned?tab=preview

Select b.GLOBALEVENTID event_id, count(a.GLOBALEVENTID) qtity_article , b.SQLDATE event_day, b.ActionGeo_Fullname event_location, a.MentionDocTranslationInfo  mention_language
FROM [gdelt-bq:gdeltv2.eventmentions_partitioned] a


JOIN EACH ( SELECT GLOBALEVENTID, SQLDATE, ActionGeo_Fullname
FROM [gdelt-bq:gdeltv2.events_partitioned] 
WHERE
SQLDATE>=20160601 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND SQLDATE<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")
)b

on a.GLOBALEVENTID = b.GLOBALEVENTID


WHERE MentionTimeDate>=20160601000000 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND MentionTimeDate<=20160610999999 and _PARTITIONTIME <= TIMESTAMP("2016-06-10")


GROUP BY b.GLOBALEVENTID, b.SQLDATE, b.ActionGeo_Fullname, a.MentionDocTranslationInfo 

ORDER BY qtity_article

_______________________________________________

Query 
-----------------
Select b.GLOBALEVENTID event_id, count(a.GLOBALEVENTID) qtity_article , b.SQLDATE event_day, b.ActionGeo_Fullname event_location, a.MentionDocTranslationInfo  mention_language

FROM EVENTMENTIONS_TABLE a
JOIN EACH ( SELECT GLOBALEVENTID, SQLDATE, ActionGeo_Fullname
FROM EVENTS_TABLE )b

on a.GLOBALEVENTID = b.GLOBALEVENTID


GROUP BY b.GLOBALEVENTID, b.SQLDATE, b.ActionGeo_Fullname, a.MentionDocTranslationInfo 

ORDER BY qtity_article




- on peut essayer de comparer numMention/numArticle from events with  count(\*) from eventmention 

- pour recuperer la langue de l'article , colonne MentionDocTranslationInfo    >> Faut il une solution pour extraire la langue apres srclc ?  ex : (srclc:fra) et faut - il remplacer la valeur "null" par english?


- (remplacer SQLDATE par Day pour eventstable en dehors de Google BigQuery)


### 2) pour un pays donné en paramètre, affichez les évènements qui y ont eu place triées par le nombre de mentions (tri décroissant); permettez une agrégation par jour/mois/année

GOOGLE QUERY 
-----------------
SELECT b.GLOBALEVENTID event_id, count(a.GLOBALEVENTID) qtity_article,  b.SQLDATE jour , b.MonthYear mois, b.Year annee, b.ActionGeo_CountryCode pays

FROM [gdelt-bq:gdeltv2.eventmentions_partitioned] a
JOIN EACH ( SELECT GLOBALEVENTID, SQLDATE, MonthYear, Year , ActionGeo_CountryCode
FROM [gdelt-bq:gdeltv2.events_partitioned] 
WHERE
SQLDATE>=20160601 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND SQLDATE<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")
)b

on a.GLOBALEVENTID = b.GLOBALEVENTID


WHERE 
MentionTimeDate>=20160601000000 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND MentionTimeDate<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")

AND b.ActionGeo_CountryCode = "US"


GROUP BY event_id, jour, mois, annee, pays

ORDER BY qtity_article DESC

_______________________________________________

Query 
----------------

SELECT b.GLOBALEVENTID event_id, count(a.GLOBALEVENTID) qtity_article,  b.Day, b.MonthYear, b.Year, b.ActionGeo_CountryCode pays

FROM EVENTMENTIONS_TABLE a

JOIN EACH (SELECT GLOBALEVENTID, SQLDATE, MonthYear, Year , ActionGeo_CountryCode
FROM EVENTS_TABLE)b

on a.GLOBALEVENTID = b.GLOBALEVENTID

WHERE b.ActionGeo_CountryCode = "US"

GROUP BY event_id, jour, mois, annee, pays

ORDER BY qtity_article DESC


- ActionGeo_FullName or  ActionGeo_CountryCode

- peut être qu'on peut aggréger par jour, mois et année directement avec la colonne SQLDATE (avec une commande du type "like" suivi d'une REGEX  et supprimer les colonnes MonthYear et Year

### 3) pour une source de donnés passée en paramètre (gkg.SourceCommonName) affichez les thèmes, personnes, lieux dont les articles de cette sources parlent ainsi que le le nombre d’articles et le ton moyen des articles (pour chaque thème/personne/lieu); permettez une agrégation par jour/mois/année.

utiliser la table GKG


V2Person; V2themes; V2Location

SELECT aggregate
Themes,Location,Persons, V2Tone

SELECT b.GLOBALEVENTID event_id, count(a.GLOBALEVENTID) qtity_article,  b.SQLDATE jour , b.MonthYear mois, b.Year annee, b.ActionGeo_CountryCode pays

FROM [gdelt-bq:gdeltv2.eventmentions_partitioned] a

JOIN EACH ( SELECT GLOBALEVENTID, SQLDATE, MonthYear, Year , 
FROM [gdelt-bq:gdeltv2.events_partitioned] 
WHERE
SQLDATE>=20160601 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND SQLDATE<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")
)b

on a.GLOBALEVENTID = b.GLOBALEVENTID

WHERE 
MentionTimeDate>=20160601000000 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND MentionTimeDate<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")



WHERE 
MentionTimeDate>=20160601000000 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND MentionTimeDate<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")

AND SourceCommonName = "cbc.ca"


GROUP BY event_id, jour, mois, annee, pays

ORDER BY qtity_article DESC

and DATE>=20160601000000 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND DATE<=20160610999999 and _PARTITIONTIME <= TIMESTAMP("2016-06-10")


SOURCE



### 4) dresser la cartographie des relations entre les pays d’après le ton des articles : pour chaque paire (pays1, pays2), calculer le nombre d’article, le ton moyen (aggrégations sur Année/Mois/Jour, filtrage par pays ou carré de coordonnées)


SELECT b.Actor1CountryCode Pays1, b.Actor2CountryCode Pays2,  count(a.GLOBALEVENTID) qtity_article, SUM(MentionDocTone)/COUNT(MentionDocTone) ton_moyen

FROM [gdelt-bq:gdeltv2.eventmentions_partitioned] a


JOIN EACH ( SELECT GLOBALEVENTID, Actor1CountryCode, Actor2CountryCode 
FROM [gdelt-bq:gdeltv2.events_partitioned] 
WHERE
SQLDATE>=20160601 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND SQLDATE<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")

AND Actor1CountryCode = "USA" and Actor2CountryCode = "FRA"

)b

on a.GLOBALEVENTID = b.GLOBALEVENTID


WHERE 
MentionTimeDate>=20160601000000 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND MentionTimeDate<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")

GROUP BY Pays1, Pays2

WHERE GLOBALEVENTID in 

SELECT GLOBALEVENTID
FROM [gdelt-bq:gdeltv2.events_partitioned] 
WHERE
SQLDATE>=20160601 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND SQLDATE<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")

AND Actor1CountryCode = "USA" and Actor2CountryCode = "FRA"

)b

Union

SELECT GLOBALEVENTID
FROM [gdelt-bq:gdeltv2.events_partitioned] 
WHERE
SQLDATE>=20160601 and _PARTITIONTIME >= TIMESTAMP("2016-06-01") 
AND SQLDATE<=20160605 and _PARTITIONTIME <= TIMESTAMP("2016-06-05")

AND Actor1CountryCode = "FRA" and Actor2CountryCode = "USA"

)b


### problème observé


probleme de Latence avec S3
ordre de vitesse : disque locaux , puis EBS et enfin S"
